<h1>After Scraping: Cleaning and Feature Engineering</h1>

- Acknowledgements:
    - ufcstats for comprehensive data sets on past MMA bouts: http://ufcstats.com/
    - Rajeev Warrier for providing the groundwork for this prediction project: https://github.com/WarrierRajeev/UFC-Predictions

In [792]:
import pandas as pd
import numpy as np

DATA_PATH ='./data'
df_fighters = pd.read_csv(DATA_PATH+'/fighter_details.csv')
df_fights = pd.read_csv(DATA_PATH+'/total_fight_data.csv', sep=';')

In [793]:
df_fighters.head(3)

,fighter_name,Height,Weight,Reach,Stance,DOB
0,Tom Aaron,NaN,155 lbs.,NaN,NaN,"Jul 13, 1978"
1,Danny Abbadi,"5' 11""",155 lbs.,NaN,Orthodox,"Jul 03, 1983"
2,David Abbott,"6' 0""",265 lbs.,NaN,Switch,NaN


In [794]:
df_fights.head(3)

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Kevin Lee,Charles Oliveira,0,0,41 of 80,43 of 65,51%,66%,61 of 100,51 of 73,...,6 of 7,Submission,3,0:28,5 Rnd (5-5-5-5-5),Mike Beltran,"March 14, 2020","Brasilia, Distrito Federal, Brazil",Lightweight Bout,Charles Oliveira
1,Demian Maia,Gilbert Burns,0,1,4 of 7,13 of 16,57%,81%,4 of 7,14 of 17,...,8 of 9,KO/TKO,1,2:34,3 Rnd (5-5-5),Osiris Maia,"March 14, 2020","Brasilia, Distrito Federal, Brazil",Welterweight Bout,Gilbert Burns
2,Renato Moicano,Damir Hadzovic,0,0,1 of 2,1 of 5,50%,20%,4 of 5,1 of 5,...,0 of 0,Submission,1,0:44,3 Rnd (5-5-5),Eduardo Herdy,"March 14, 2020","Brasilia, Distrito Federal, Brazil",Lightweight Bout,Renato Moicano


<h3>Processing Fighter data set</h3> 

In [795]:
df_fighters.isna().sum()

fighter_name       0
Height           257
Weight            74
Reach           1714
Stance           786
DOB              739
dtype: int64

- fighters with NaN Weight values have little to no useful data
    - therefore, these rows will be excluded

In [796]:
df_fighters[pd.isnull(df_fighters['Weight'])].isna().sum()

fighter_name     0
Height          68
Weight          74
Reach           74
Stance          65
DOB             72
dtype: int64

In [797]:
df_fighters = df_fighters[df_fighters['Weight'].notna()]

- to fill NaN values in bodily metrics, find:
    - average reach for each height increment
    - average height for each weight increment

In [798]:
df_fighters['Weight'] = df_fighters['Weight'].apply(lambda x: x.split(' ')[0])
df_fighters['Weight'] = df_fighters['Weight'].astype(float)

In [799]:
df_fighters['Height'] = df_fighters['Height'].fillna('0\' 0\"')
df_fighters['Height'] = df_fighters['Height'].apply(lambda x: int(x.split('\' ')[0])*12 + int(x.split('\' ')[1].replace('\"','')))
df_fighters['Height'] = df_fighters['Height'].replace(0, np.nan).astype(float)

In [800]:
df_fighters['Height'] = df_fighters.groupby('Weight')['Height'].apply(lambda x: x.fillna(x.mean()))
df_fighters['Height'] = df_fighters['Height'].fillna(df_fighters['Height'].mean())

In [801]:
df_fighters['Reach'] = df_fighters['Reach'].fillna('0')
df_fighters['Reach'] = df_fighters['Reach'].apply(lambda x: x.replace('\"',''))
df_fighters['Reach'] = df_fighters['Reach'].replace('0', np.nan).astype(float)

In [802]:
df_fighters['Reach'] = df_fighters.groupby('Height')['Reach'].apply(lambda x: x.fillna(x.mean()))
df_fighters['Reach'] = df_fighters['Reach'].fillna(df_fighters['Reach'].mean())

In [803]:
df_fighters['Stance'].value_counts()

Orthodox       2047
Southpaw        460
Switch          100
Open Stance       7
Sideways          3
Name: Stance, dtype: int64

<h3>Processing Fight data set</h3>

- split attack stats into attempts/landed numerical format

In [804]:
df_fights.columns
attack_cols = ['R_SIG_STR.', 'B_SIG_STR.','R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND']

In [805]:
for col in attack_cols:
    df_fights[col+'_ATT'] = df_fights[col].apply(lambda x: int(x.split('of')[1]))
    df_fights[col+'_LANDED'] = df_fights[col].apply(lambda x: int(x.split('of')[0]))

In [806]:
df_fights.drop(attack_cols, axis=1, inplace=True)

- check for NULL values

In [807]:
for col in df_fights:
    if df_fights[col].isnull().sum()!=0:
        print(f'Null count in {col} = {df_fights[col].isnull().sum()}')

Null count in Referee = 25
Null count in Winner = 94


In [808]:
df_fights[df_fights['Winner'].isnull()]['win_by'].value_counts()

Overturned              38
Decision - Majority     23
Could Not Continue      15
Decision - Split        11
Decision - Unanimous     5
Other                    2
Name: win_by, dtype: int64

In [809]:
df_fights['Winner'].fillna('Draw', inplace=True)

- convert percentages to decimal values

In [810]:
percentage_columns = ['R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct']

for col in percentage_columns:
    df_fights[col] = df_fights[col].apply(lambda x : float(x.replace('%',''))/100)

- isolating Title fights and weight classes

In [811]:
df_fights['Fight_type'].value_counts()[df_fights['Fight_type'].value_counts() > 1].index

Index(['Lightweight Bout', 'Welterweight Bout', 'Middleweight Bout',
       'Light Heavyweight Bout', 'Heavyweight Bout', 'Featherweight Bout',
       'Bantamweight Bout', 'Flyweight Bout', 'Women's Strawweight Bout',
       'Women's Bantamweight Bout', 'Open Weight Bout',
       'Women's Flyweight Bout', 'UFC Light Heavyweight Title Bout',
       'UFC Welterweight Title Bout', 'UFC Heavyweight Title Bout',
       'UFC Middleweight Title Bout', 'UFC Lightweight Title Bout',
       'Catch Weight Bout', 'UFC Flyweight Title Bout',
       'UFC Women's Bantamweight Title Bout', 'UFC Featherweight Title Bout',
       'UFC Bantamweight Title Bout', 'UFC Women's Strawweight Title Bout',
       'Women's Featherweight Bout', 'UFC Interim Heavyweight Title Bout',
       'UFC Women's Featherweight Title Bout',
       'UFC Women's Flyweight Title Bout', 'UFC Superfight Championship Bout',
       'UFC Interim Featherweight Title Bout',
       'UFC Interim Welterweight Title Bout',
       'UFC Inter

In [812]:
df_fights['title_bout'] = df_fights['Fight_type'].apply(lambda x: 1 if 'Title Bout' in x else 0) 

In [813]:
weight_classes = ['Women\'s Strawweight', 'Women\'s Bantamweight', 
                  'Women\'s Featherweight', 'Women\'s Flyweight', 'Lightweight', 
                  'Welterweight', 'Middleweight','Light Heavyweight', 
                  'Heavyweight', 'Featherweight','Bantamweight', 'Flyweight', 'Open Weight']

def make_weight_class(x):
    for weight_class in weight_classes:
        if weight_class in x:
            return weight_class
    if x == 'Catch Weight Bout' or 'Catchweight Bout':
        return 'Catch Weight'
    else:
        return 'Open Weight'

In [814]:
df_fights['weight_class'] = df_fights['Fight_type'].apply(make_weight_class)

In [815]:
df_fights['weight_class'].value_counts()

Lightweight              1043
Welterweight             1027
Middleweight              763
Heavyweight               539
Light Heavyweight         536
Featherweight             488
Bantamweight              422
Flyweight                 206
Women's Strawweight       165
Women's Bantamweight      130
Open Weight                93
Women's Flyweight          78
Catch Weight               39
Women's Featherweight      14
Name: weight_class, dtype: int64

- isolate total fight time (seconds)

In [816]:
df_fights['Format'].value_counts()

3 Rnd (5-5-5)           4860
5 Rnd (5-5-5-5-5)        459
1 Rnd + OT (12-3)         80
No Time Limit             37
3 Rnd + OT (5-5-5-5)      22
1 Rnd (20)                21
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                 8
1 Rnd (10)                 6
1 Rnd (12)                 4
1 Rnd + OT (30-5)          3
1 Rnd + OT (15-3)          2
1 Rnd (18)                 2
1 Rnd + 2OT (24-3-3)       1
1 Rnd (30)                 1
1 Rnd + OT (30-3)          1
1 Rnd + OT (27-3)          1
1 Rnd + OT (31-5)          1
Name: Format, dtype: int64

In [817]:
time_in_first_round = {'3 Rnd (5-5-5)': 5*60, 
                       '5 Rnd (5-5-5-5-5)': 5*60, 
                       '1 Rnd + OT (12-3)': 12*60,
                       'No Time Limit': 1, 
                       '3 Rnd + OT (5-5-5-5)': 5*60, 
                       '1 Rnd (20)': 1*20,
                       '2 Rnd (5-5)': 5*60, 
                       '1 Rnd (15)': 15*60, 
                       '1 Rnd (10)': 10*60,
                       '1 Rnd (12)':12*60, 
                       '1 Rnd + OT (30-5)': 30*60, 
                       '1 Rnd (18)': 18*60, 
                       '1 Rnd + OT (15-3)': 15*60,
                       '1 Rnd (30)': 30*60, 
                       '1 Rnd + OT (31-5)': 31*5,
                       '1 Rnd + OT (27-3)': 27*60, 
                       '1 Rnd + OT (30-3)': 30*60}

exception_format_time = {'1 Rnd + 2OT (15-3-3)': [15*60, 3*60], 
                         '1 Rnd + 2OT (24-3-3)': [24*60, 3*60]}

In [818]:
# Converting to seconds
df_fights['last_round_time'] = df_fights['last_round_time'].apply(lambda x: int(x.split(':')[0])*60 + int(x.split(':')[1]))

In [819]:
def get_total_time(row):
    if row['Format'] in time_in_first_round.keys():
        return (row['last_round'] - 1) * time_in_first_round[row['Format']] + row['last_round_time']
    elif row['Format'] in exception_format_time.keys():
        if (row['last_round'] - 1) >= 2:
            return exception_format_time[row['Format']][0] + (row['last_round'] - 2) * \
                    exception_format_time[row['Format']][1] + row['last_round_time']
        else:
            return (row['last_round'] - 1) * exception_format_time[row['Format']][0] + row['last_round_time']

In [820]:
df_fights['total_time_fought(sec)'] = df_fights.apply(get_total_time, axis=1)

In [821]:
def get_num_rounds(x):
    if x == 'No Time Limit':
        return 1
    else:
        return len((x.split('(')[1].replace(')','').split('-')))
    
df_fights['no_of_rounds'] = df_fights['Format'].apply(get_num_rounds)

- there are too many distinct locations
    - in order to create a more signifcant feature, location is adapted to a binary indicator of whether or not the fight took place in Las Vegas, Nevada (i.e. the most popular fight location)

In [822]:
df_fights['location'].value_counts()

Las Vegas, Nevada, USA             1264
London, England, United Kingdom     114
Newark, New Jersey, USA              90
Anaheim, California, USA             84
Chicago, Illinois, USA               81
                                   ... 
San Juan, Puerto Rico                 8
Miami, Florida, USA                   8
Chiba, Japan                          7
Dothan, Alabama, USA                  7
Yokohama, Kanagawa, Japan             7
Name: location, Length: 165, dtype: int64

In [823]:
df_fights['location']=df_fights['location'].apply(lambda x: 1 if str(x).find('Las Vegas')!=-1 else 0)

- change Date of Birth and fight date from string to datetime

In [824]:
from datetime import datetime

month_code = {'Jan ': 'January ', 
      'Feb ': 'February ', 
      'Mar ': 'March ', 
      'Apr ': 'April ', 
      'May ': 'May ', 
      'Jun ': 'June ', 
      'Jul ': 'July ', 
      'Aug ': 'August ', 
      'Sep ': 'September ', 
      'Oct ': 'October ', 
      'Nov ': 'November ', 
      'Dec ': 'December '}

for k, v in month_code.items():
    df_fighters['DOB'] = df_fighters['DOB'].apply(lambda x: x.replace(k, v) if type(x) == str else x)

df_fighters['DOB'] = df_fighters['DOB'].apply(lambda row: datetime.strptime(row, '%B %d, %Y') if type(row) == str else row)
df_fights['date'] = df_fights['date'].apply(lambda row: datetime.strptime(row, '%B %d, %Y') if type(row) == str else row)

- recode winner column to binary and drop obsolete columns

In [825]:
df_fights['Red_win'] = df_fights.apply(lambda row: 1 if row['Winner'] == row['R_fighter'] else 0, axis=1)

df_fights.drop(columns = ['Format', 'Referee','Fight_type','last_round_time'], inplace=True)

- recode win_by feature into bins for Submission, KO, or Other

In [826]:
df_fights['win_by'].value_counts()

Decision - Unanimous       1903
KO/TKO                     1763
Submission                 1136
Decision - Split            533
TKO - Doctor's Stoppage      74
Decision - Majority          62
Overturned                   38
DQ                           17
Could Not Continue           15
Other                         2
Name: win_by, dtype: int64

In [827]:
df_fights['win_by'] = df_fights.apply(lambda row: 'Submission' if 'Submission' in row['win_by']
                                                  else('KO' if 'KO' in row['win_by']
                                                  else 'Other'), axis=1)

<h3>Consolidate red/blue corner stats to align them with the correct fighter</h3>

In [828]:
df_red = df_fights[['R_fighter','R_KD', 'R_SIG_STR_pct',
       'R_TD_pct', 'R_SUB_ATT',
       'R_PASS', 'R_REV', 
       'R_SIG_STR._ATT', 'R_SIG_STR._LANDED',
       'R_TOTAL_STR._ATT',
       'R_TOTAL_STR._LANDED',
       'R_TD_ATT', 'R_TD_LANDED', 'R_HEAD_ATT',
       'R_HEAD_LANDED', 'R_BODY_ATT',
       'R_BODY_LANDED',  'R_LEG_ATT',
       'R_LEG_LANDED',  'R_DISTANCE_ATT',
       'R_DISTANCE_LANDED', 
       'R_CLINCH_ATT', 'R_CLINCH_LANDED',
       'R_GROUND_ATT', 'R_GROUND_LANDED',
       'Winner', 'win_by', 'last_round',
       'date', 'location',
       'title_bout', 'weight_class', 'total_time_fought(sec)', 'no_of_rounds']]

df_blue = df_fights[['B_fighter',  'B_KD',
       'B_SIG_STR_pct','B_TD_pct', 'B_SUB_ATT',
       'B_PASS',  'B_REV', 
       'B_SIG_STR._ATT', 'B_SIG_STR._LANDED',
       'B_TOTAL_STR._ATT', 'B_TOTAL_STR._LANDED',
       'B_TD_ATT', 'B_TD_LANDED',
       'B_HEAD_ATT', 'B_HEAD_LANDED', 
       'B_BODY_ATT', 'B_BODY_LANDED', 
       'B_LEG_ATT', 'B_LEG_LANDED', 
       'B_DISTANCE_ATT', 'B_DISTANCE_LANDED',
       'B_CLINCH_ATT', 'B_CLINCH_LANDED',
       'B_GROUND_ATT', 'B_GROUND_LANDED',
       'Winner', 'win_by', 'last_round',
       'date', 'location',
       'title_bout', 'weight_class', 'total_time_fought(sec)', 'no_of_rounds']]

- get rid of red/blue corner prefixes in order to union fighter history

In [829]:
def drop_prefix(self, prefix):
    self.columns = self.columns.str.replace('^'+prefix,'')
    return self

pd.core.frame.DataFrame.drop_prefix = drop_prefix

In [830]:
union = pd.concat([df_red.drop_prefix('R_'), df_blue.drop_prefix('B_')])

- join this combined fight history DataFrame to the originial fighter DataFrame

In [831]:
union[union['fighter']=='Daniel Cormier'].head(3)

,fighter,KD,SIG_STR_pct,TD_pct,SUB_ATT,PASS,REV,SIG_STR._ATT,SIG_STR._LANDED,TOTAL_STR._ATT,...,GROUND_LANDED,Winner,win_by,last_round,date,location,title_bout,weight_class,total_time_fought(sec),no_of_rounds
280,Daniel Cormier,0,0.68,0.33,0,2,0,263,181,317,...,21,Stipe Miocic,KO,4,2019-08-17,0,1,Heavyweight,1149,5
710,Daniel Cormier,0,0.76,1.00,1,4,0,25,19,54,...,14,Daniel Cormier,Submission,2,2018-11-03,0,1,Heavyweight,434,5
1064,Daniel Cormier,0,0.52,0.66,1,3,0,61,32,113,...,5,Daniel Cormier,KO,2,2018-01-20,0,1,Light Heavyweight,420,5


In [832]:
df_fighter_history = pd.merge(df_fighters, union, left_on='fighter_name', right_on='fighter', how='left', indicator=True)

- 1,330 fighters without any fight stats (in original fighter dataset)
    - However, every fighter involved in a historical bout is contained in the original fighter dataset
    - UPDATE: after analysis using the above 1,330 fighters, they will be dropped to ensure data quality and avoid "garbage in, garbage out

In [833]:
df_fighter_history._merge.value_counts()

both          11076
left_only      1330
right_only        0
Name: _merge, dtype: int64

In [834]:
df_fighter_history = df_fighter_history[df_fighter_history._merge != 'left_only']

In [835]:
union.shape

(11086, 34)

- replace categorical feature (i.e. Stance) NULLs with the mode of that column

In [836]:
df_fighter_history['Stance'].fillna(df_fighter_history['Stance'].value_counts().index[0], inplace=True)

In [837]:
df_fighter_history.shape

(11076, 41)

- lack of depth in individual fight history presents a problem for forecasting fighter performance

In [838]:
df_fighter_history['fighter_name'].value_counts()

Jim Miller          34
Donald Cerrone      34
Jeremy Stephens     32
Demian Maia         32
Diego Sanchez       31
                    ..
Dieusel Berto        1
Liana Jojua          1
Fredson Paixao       1
Daniel Rodriguez     1
Delson Heleno        1
Name: fighter_name, Length: 2008, dtype: int64

<h3>Feature Engineering</h3>

In [839]:
df_fights[df_fights['B_fighter']=='Omar Morales']

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,...,B_CLINCH_LANDED,R_GROUND_ATT,R_GROUND_LANDED,B_GROUND_ATT,B_GROUND_LANDED,title_bout,weight_class,total_time_fought(sec),no_of_rounds,Red_win
103,Dong Hyun Ma,Omar Morales,0,1,0.26,0.35,0.0,1.0,0,0,...,0,0,0,20,12,0,Lightweight,900,3,0


In [840]:
df_fighter_history.head()

,fighter_name,Height,Weight,Reach,Stance,DOB,fighter,KD,SIG_STR_pct,TD_pct,...,Winner,win_by,last_round,date,location,title_bout,weight_class,total_time_fought(sec),no_of_rounds,_merge
1,Danny Abbadi,71.0,155.0,72.68125,Orthodox,1983-07-03,Danny Abbadi,0.0,0.38,0.00,...,Jorge Gurgel,Other,3.0,2006-09-23,0.0,0.0,Lightweight,900.0,3.0,both
2,Danny Abbadi,71.0,155.0,72.68125,Orthodox,1983-07-03,Danny Abbadi,0.0,0.33,0.00,...,Kalib Starnes,Submission,1.0,2006-06-24,1.0,0.0,Middleweight,176.0,3.0,both
3,David Abbott,72.0,265.0,73.75000,Switch,NaT,David Abbott,0.0,0.68,0.00,...,David Abbott,KO,1.0,1998-05-15,0.0,0.0,Heavyweight,43.0,2.0,both
4,David Abbott,72.0,265.0,73.75000,Switch,NaT,David Abbott,0.0,0.41,0.75,...,David Abbott,Other,2.0,1997-12-21,0.0,0.0,Heavyweight,900.0,2.0,both
5,David Abbott,72.0,265.0,73.75000,Switch,NaT,David Abbott,1.0,0.52,1.00,...,David Abbott,KO,1.0,1996-12-07,0.0,0.0,Open Weight,63.0,2.0,both


- creating age (at fight date) feature

In [841]:
df_fighter_history['age'] = df_fighter_history['date'] - df_fighter_history['DOB']
df_fighter_history['age']=df_fighter_history['age']/np.timedelta64(1,'Y')
df_fighter_history['age'].fillna(df_fighter_history['age'].mean(), inplace=True)

In [842]:
df_fighter_history['age'].sort_values()

2453     18.267316
6076     18.541106
8132     19.589725
8131     19.775902
5226     19.784116
           ...    
2197     46.399310
2196     46.629294
2195     47.185089
2211     47.855877
11657    51.891552
Name: age, Length: 11076, dtype: float64

In [843]:
df_fighter_history['age'].mean()

29.848766940940827

In [844]:
df_fighter_history.drop(columns='_merge', inplace=True)

- create features for 1) # of fights they've been in, 2) what % they won, and 3) the ranked order of past fights

In [845]:
df_fighter_history['num_fights'] = df_fighter_history['date'].groupby(df_fighter_history['fighter_name']).transform('count')

df_fighter_history['win'] = df_fighter_history.apply(lambda row: 1 if row['Winner'] == row['fighter_name'] else 0, axis=1)
df_fighter_history.drop(columns=['Winner'], inplace=True)

df_fighter_history['num_wins'] = df_fighter_history['win'].groupby(df_fighter_history['fighter_name']).transform('sum')

df_fighter_history['record'] = df_fighter_history['num_wins']/df_fighter_history['num_fights']

In [846]:
df_fighter_history['title_bout']=df_fighter_history['title_bout'].apply(lambda x: 1 if x == 1 else 0)

In [847]:
df_fighter_history['fight_rank']=df_fighter_history.groupby('fighter_name')['date'].rank(ascending=True, method='first')

In [848]:
df_fighter_history.drop(columns='fighter', inplace=True)

In [849]:
df_fights_train = df_fights[['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_PASS', 'B_PASS', 'R_REV', 'B_REV', 'win_by', 'last_round', 'date',
       'location', 'R_SIG_STR._ATT', 'R_SIG_STR._LANDED',
       'B_SIG_STR._ATT', 'B_SIG_STR._LANDED', 'R_TOTAL_STR._ATT',
       'R_TOTAL_STR._LANDED', 'B_TOTAL_STR._ATT', 'B_TOTAL_STR._LANDED',
       'R_TD_ATT', 'R_TD_LANDED', 'B_TD_ATT', 'B_TD_LANDED', 'R_HEAD_ATT',
       'R_HEAD_LANDED', 'B_HEAD_ATT', 'B_HEAD_LANDED', 'R_BODY_ATT',
       'R_BODY_LANDED', 'B_BODY_ATT', 'B_BODY_LANDED', 'R_LEG_ATT',
       'R_LEG_LANDED', 'B_LEG_ATT', 'B_LEG_LANDED', 'R_DISTANCE_ATT',
       'R_DISTANCE_LANDED', 'B_DISTANCE_ATT', 'B_DISTANCE_LANDED',
       'R_CLINCH_ATT', 'R_CLINCH_LANDED', 'B_CLINCH_ATT', 'B_CLINCH_LANDED',
       'R_GROUND_ATT', 'R_GROUND_LANDED', 'B_GROUND_ATT', 'B_GROUND_LANDED',
       'title_bout', 'weight_class', 'total_time_fought(sec)', 'no_of_rounds',
       'Red_win']]

In [850]:
df_fighter_history_train = df_fighter_history[['fighter_name', 'Height', 
                            'Weight', 'Reach', 'Stance', 'DOB', 'win_by',
                            'date',
                           'win']]

In [851]:
df_fighter_history_train.head(3)

,fighter_name,Height,Weight,Reach,Stance,DOB,win_by,date,win
1,Danny Abbadi,71.0,155.0,72.68125,Orthodox,1983-07-03,Other,2006-09-23,0
2,Danny Abbadi,71.0,155.0,72.68125,Orthodox,1983-07-03,Submission,2006-06-24,0
3,David Abbott,72.0,265.0,73.75000,Switch,NaT,KO,1998-05-15,1


- first merge for red fighter

In [852]:
df_train = pd.merge(df_fights_train, df_fighter_history_train, left_on='R_fighter',right_on='fighter_name',sort=False)

- for each bout, filter to only previous fights

In [853]:
df_train = df_train[df_train['date_x'] > df_train['date_y']]
df_train.drop(columns=['date_y','fighter_name'], inplace=True)

- create dummy variables for fighter-specific categorical variables (i.e. stance, win_by)

In [854]:
df_train = pd.concat([df_train, pd.get_dummies(df_train['win_by_y'])], axis=1)
df_train.drop(columns=['win_by_y','Other'], inplace=True)
df_train.rename(columns={'date_x':'date', 'KO':'R_KO_win_%', 'Submission':'R_Sub_win_%'}, inplace=True)

df_train = pd.concat([df_train, pd.get_dummies(df_train['Stance'])], axis=1)
df_train.drop(columns=['Stance','Switch','Open Stance','Sideways'], inplace=True)
df_train.rename(columns={'Orthodox':'R_Stance_Orthodox', 
                         'Southpaw':'R_Stance_Southpaw',
                         'Height':'R_Height',
                         'Weight':'R_Weight',
                         'Reach':'R_Reach'}, inplace=True)

- recalculate number of past fights, fighter record, and fighter age

In [855]:
df_train['R_num_fights'] = df_train.groupby(['R_fighter','date'])['date'].transform('count')

df_train['R_num_wins'] = df_train.groupby(['R_fighter','date'])['win'].transform('sum')

df_train['R_record'] = df_train['R_num_wins']/df_train['R_num_fights']

df_train.drop(columns=['win','R_num_wins'], inplace=True)

In [856]:
df_train['R_age'] = df_train['date'] - df_train['DOB']
df_train['R_age']=df_train['R_age']/np.timedelta64(1,'Y')
df_train['R_age'].fillna(df_train['R_age'].mean(), inplace=True)

df_train.drop(columns=['DOB'], inplace=True)

In [857]:
df_train[['R_KO_win_%', 'R_Sub_win_%']] = df_train.groupby(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_PASS', 'B_PASS', 'R_REV', 'B_REV', 'win_by_x', 'last_round', 'date',
       'location', 'R_SIG_STR._ATT', 'R_SIG_STR._LANDED', 'B_SIG_STR._ATT',
       'B_SIG_STR._LANDED', 'R_TOTAL_STR._ATT', 'R_TOTAL_STR._LANDED',
       'B_TOTAL_STR._ATT', 'B_TOTAL_STR._LANDED', 'R_TD_ATT', 'R_TD_LANDED',
       'B_TD_ATT', 'B_TD_LANDED', 'R_HEAD_ATT', 'R_HEAD_LANDED', 'B_HEAD_ATT',
       'B_HEAD_LANDED', 'R_BODY_ATT', 'R_BODY_LANDED', 'B_BODY_ATT',
       'B_BODY_LANDED', 'R_LEG_ATT', 'R_LEG_LANDED', 'B_LEG_ATT',
       'B_LEG_LANDED', 'R_DISTANCE_ATT', 'R_DISTANCE_LANDED', 'B_DISTANCE_ATT',
       'B_DISTANCE_LANDED', 'R_CLINCH_ATT', 'R_CLINCH_LANDED', 'B_CLINCH_ATT',
       'B_CLINCH_LANDED', 'R_GROUND_ATT', 'R_GROUND_LANDED', 'B_GROUND_ATT',
       'B_GROUND_LANDED', 'title_bout', 'weight_class',
       'total_time_fought(sec)', 'no_of_rounds', 'Red_win', 'R_Height', 'R_Weight',
       'R_Reach', 'R_Stance_Orthodox',
       'R_Stance_Southpaw', 'R_num_fights', 'R_record', 'R_age'])['R_KO_win_%', 'R_Sub_win_%'].transform('mean')

df_train = df_train.drop_duplicates()

- repeat steps for blue fighter

In [858]:
#merge blue fighters
df_train = pd.merge(df_train, df_fighter_history_train, left_on='B_fighter',right_on='fighter_name',sort=False)

#only past fights
df_train = df_train[df_train['date_x'] > df_train['date_y']]
df_train.drop(columns=['date_y','fighter_name'], inplace=True)

#dummy variables
df_train = pd.concat([df_train, pd.get_dummies(df_train['win_by'])], axis=1)
df_train.drop(columns=['win_by','Other'], inplace=True)
df_train.rename(columns={'date_x':'date', 'KO':'B_KO_win_%', 'Submission':'B_Sub_win_%'}, inplace=True)

df_train = pd.concat([df_train, pd.get_dummies(df_train['Stance'])], axis=1)
df_train.drop(columns=['Stance','Switch','Open Stance','Sideways'], inplace=True)
df_train.rename(columns={'Orthodox':'B_Stance_Orthodox', 
                         'Southpaw':'B_Stance_Southpaw',
                         'Height':'B_Height',
                         'Weight':'B_Weight',
                         'Reach':'B_Reach'}, inplace=True)

#num_fights and record
df_train['B_num_fights'] = df_train.groupby(['B_fighter','date'])['date'].transform('count')

df_train['B_num_wins'] = df_train.groupby(['B_fighter','date'])['win'].transform('sum')

df_train['B_record'] = df_train['B_num_wins']/df_train['B_num_fights']

df_train.drop(columns=['win','B_num_wins'], inplace=True)

#age
df_train['B_age'] = df_train['date'] - df_train['DOB']
df_train['B_age']=df_train['B_age']/np.timedelta64(1,'Y')
df_train['B_age'].fillna(df_train['B_age'].mean(), inplace=True)

df_train.drop(columns=['DOB'], inplace=True)

#win_by percentages
df_train[['B_KO_win_%', 'B_Sub_win_%']] = df_train.groupby(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_PASS', 'B_PASS', 'R_REV', 'B_REV', 'win_by_x', 'last_round', 'date',
       'location', 'R_SIG_STR._ATT', 'R_SIG_STR._LANDED', 'B_SIG_STR._ATT',
       'B_SIG_STR._LANDED', 'R_TOTAL_STR._ATT', 'R_TOTAL_STR._LANDED',
       'B_TOTAL_STR._ATT', 'B_TOTAL_STR._LANDED', 'R_TD_ATT', 'R_TD_LANDED',
       'B_TD_ATT', 'B_TD_LANDED', 'R_HEAD_ATT', 'R_HEAD_LANDED', 'B_HEAD_ATT',
       'B_HEAD_LANDED', 'R_BODY_ATT', 'R_BODY_LANDED', 'B_BODY_ATT',
       'B_BODY_LANDED', 'R_LEG_ATT', 'R_LEG_LANDED', 'B_LEG_ATT',
       'B_LEG_LANDED', 'R_DISTANCE_ATT', 'R_DISTANCE_LANDED', 'B_DISTANCE_ATT',
       'B_DISTANCE_LANDED', 'R_CLINCH_ATT', 'R_CLINCH_LANDED', 'B_CLINCH_ATT',
       'B_CLINCH_LANDED', 'R_GROUND_ATT', 'R_GROUND_LANDED', 'B_GROUND_ATT',
       'B_GROUND_LANDED', 'title_bout', 'weight_class',
       'total_time_fought(sec)', 'no_of_rounds', 'Red_win', 'R_Height',
       'R_Weight', 'R_Reach', 'R_KO_win_%', 'R_Sub_win_%', 'R_Stance_Orthodox',
       'R_Stance_Southpaw', 'R_num_fights', 'R_record', 'R_age', 'B_Height',
       'B_Weight', 'B_Reach', 'B_Stance_Orthodox',
       'B_Stance_Southpaw', 'B_num_fights', 'B_record', 'B_age'])['B_KO_win_%', 'B_Sub_win_%'].transform('mean')

df_train = df_train.drop_duplicates()

In [876]:
df_train[df_train['R_fighter']=='Jon Jones']

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,...,Flyweight,Heavyweight,Light Heavyweight,Lightweight,Middleweight,Welterweight,Women's Bantamweight,Women's Featherweight,Women's Flyweight,Women's Strawweight
2198,Jon Jones,Chael Sonnen,0,0,0.74,0.40,0.50,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0
13443,Jon Jones,Dominick Reyes,0,0,0.62,0.44,0.22,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0
13485,Jon Jones,Thiago Santos,0,0,0.65,0.25,0.00,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0
13675,Jon Jones,Anthony Smith,0,0,0.74,0.54,0.37,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0
13784,Jon Jones,Alexander Gustafsson,0,0,0.57,0.26,0.20,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0
13804,Jon Jones,Alexander Gustafsson,0,0,0.61,0.38,0.09,0.12,0,0,...,0,0,1,0,0,0,0,0,0,0
13904,Jon Jones,Ovince Saint Preux,0,0,0.66,0.35,0.60,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0
14022,Jon Jones,Daniel Cormier,0,0,0.51,0.35,0.60,0.12,0,0,...,0,0,1,0,0,0,0,0,0,0
14081,Jon Jones,Glover Teixeira,0,0,0.58,0.27,0.50,0.00,2,0,...,0,0,1,0,0,0,0,0,0,0
14169,Jon Jones,Vitor Belfort,1,0,0.59,0.44,1.00,0.00,1,1,...,0,0,1,0,0,0,0,0,0,0


- create dummy variables for weight class

In [860]:
df_train.drop(columns=['win_by_x','last_round'], inplace=True)

In [861]:
df_train = pd.concat([df_train, pd.get_dummies(df_train['weight_class'])], axis=1)
df_train.drop(columns=['weight_class','Open Weight','Catch Weight'], inplace=True)

In [874]:
df_train[df_train['R_fighter']=='Kamaru Usman']

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,...,Flyweight,Heavyweight,Light Heavyweight,Lightweight,Middleweight,Welterweight,Women's Bantamweight,Women's Featherweight,Women's Flyweight,Women's Strawweight
860,Kamaru Usman,Colby Covington,2,0,0.48,0.36,0.00,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
1684,Kamaru Usman,Alexander Yakovlev,0,0,0.61,0.53,0.55,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
4912,Kamaru Usman,Sean Strickland,1,0,0.52,0.26,0.25,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
9153,Kamaru Usman,Emil Meek,0,0,0.75,0.44,0.42,0.0,0,2,...,0,0,0,0,0,1,0,0,0,0
16572,Kamaru Usman,Leon Edwards,0,0,0.60,0.45,0.46,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
21315,Kamaru Usman,Sergio Moraes,1,0,0.33,0.48,0.00,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0


- export relevant data frames for further use

In [863]:
df_fights.to_csv(DATA_PATH+'/df_fights.csv', index = False, header=True)
df_fighter_history.to_csv(DATA_PATH+'/df_fighter_history.csv', index = False, header=True)
df_fights_train.to_csv(DATA_PATH+'/df_fights_train.csv', index = False, header=True)
df_fighter_history_train.to_csv(DATA_PATH+'/df_fighter_history_train.csv', index = False, header=True)
df_train.to_csv(DATA_PATH+'/df_train.csv', index = False, header=True)